In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.local_conv1 = nn.Conv3d(4, 64, (7, 7, 3))
        self.local_conv2 = nn.Conv3d(64, 64, (3, 3, 3))
        self.local_conv3 = nn.Conv3d(4, 160, (13, 13, 5))
        self.total_conv = nn.Conv3d(224, 5, (21, 21, 1))

    def forward(self, x):
        under_x = F.relu(self.local_conv3(x))
        x = self.local_conv1(x)
        x = F.max_pool3d(F.relu(x), (4, 4, 1), stride = 1)
        x = self.local_conv2(x)
        x = F.max_pool3d(F.relu(x), (2, 2, 1), stride = 1)
        x = torch.cat((x, under_x), 1)
        x = self.total_conv(x)
        x = x.view(-1,5)
        return x
    
def create_test_batch(img = 0, x = 16, z= 0):
    case = SAMPLE[img]
    case1 = case[:2]
    case2 = case[3:]
    batch = []
    _, X, Y, Z = train_f[case1][case2].shape
    img1 = train_f[case1][case2][:4,:,:,z]
    for y in range(16, Y - 16):
        content=img1[:, x-16:x+17, y-16:y+17]
        content[3, :,:]=0
        batch.append(content)
    batch = torch.from_numpy(np.array(batch))
    return batch